## 1. Imports and config

In [1]:
import datetime as dt
import warnings
import yaml
import logging
import pywhatkit
import pandas as pd
from pmaw import PushshiftAPI
from tqdm import tqdm
from sqlalchemy import create_engine
from data_extraction_utils import data_prep_posts, join_submission_title_and_body, find_stock_symbols, data_prep_comments
warnings.filterwarnings("ignore")

In [2]:
# Specify logging settings
logging.basicConfig(filename='comments.log', level=logging.INFO, filemode="a", format="%(asctime)s - %(levelname)s - %(message)s")

## 2. Database configuration

In [3]:
# Reading form config.yaml"
with open("../../config.yaml", "r") as yamlconfig:
    config = yaml.load(yamlconfig, Loader=yaml.FullLoader)

# Create postgres string with db-config
postgres_username = config["db_config"]["postgres_username"]
postgres_password = config["db_config"]["postgres_password"]
postgres_address = config["db_config"]["postgres_address"]
postgres_port = config["db_config"]["postgres_port"]
postgres_dbname = config["db_config"]["postgres_dbname"]

postgres_str = f"postgresql://{postgres_username}:{postgres_password}@{postgres_address}:{postgres_port}/{postgres_dbname}"

# create db connection with sqlalchemy
cnx = create_engine(postgres_str)

## 3. Create stock symbol list

In [4]:
# load stock list and transform it into a set
stock_ticker_list = pd.read_excel("../../data/external/stock_ticker_list.xlsx")["symbol"].to_list()

# Modified second stock list with dollar sign
stock_ticker_list_with_dollar_sign = [f"${ticker}" for ticker in stock_ticker_list]

# Combine the two lists to one set
final_stock_ticker_list = set(stock_ticker_list + stock_ticker_list_with_dollar_sign)

## 4. Extract Comments from reddit

In [5]:
# Set date range to extract the data from the API
start = dt.datetime.strptime("05-08-2021", "%d-%m-%Y")
end = dt.datetime.strptime("01-05-2022", "%d-%m-%Y")
timestamp_list = [int((start + dt.timedelta(days=x)).timestamp()) for x in range(0, (end-start).days +1)]

# Reddit API Object
api = PushshiftAPI(num_workers=60, batch_size=60)

In [6]:
try:
    # loop over list of timestamps
    for i in (pbar := tqdm(range(len(timestamp_list) -1))):

        # Set start date and end date
        start_date = timestamp_list[i]
        end_date = timestamp_list[i + 1]

        # Specify data to extract from reddit
        filters = ['author', 'created_utc', "score", "body"]

        # Call API function to retrieve data from reddit as a dataframe
        df = data_prep_comments(api, "wallstreetbets", start_time=start_date, end_time=end_date, filters=filters)

        # If response from API is empty -> Note that we have missing data for that day and continue
        if len(df) == 0:
            print(f"Missing Data for: {dt.datetime.fromtimestamp(start_date)}")
            logging.info(f"{df.shape[0]} rows written to DB - Date: {dt.datetime.fromtimestamp(start_date)}")
            pbar.set_description(f"{dt.datetime.fromtimestamp(start_date)}")
            continue

        # Transform timestamp into datetime column
        df["created_at"] = df["created_utc"].apply(lambda x: dt.datetime.fromtimestamp(x))

        # Get stock symbol from reddit post if available
        df["stock_symbol"] = df["body"].apply(lambda x: find_stock_symbols(x, final_stock_ticker_list))

        # Drop unused columns
        df.drop(columns=["created_utc"], inplace=True)

        # Rename columns as preparation for database
        df.rename(columns={'score': 'num_up_votes', "body": "post"}, inplace=True)

        # Write dataframe to database
        df.to_sql("r_wallstreetbets", cnx, index=False, if_exists="append")

        # Logging
        logging.info(f"{df.shape[0]} rows written to DB - Date: {dt.datetime.fromtimestamp(start_date)}")
        pbar.set_description(f"{dt.datetime.fromtimestamp(start_date)}")

except Exception as e:
    now = dt.datetime.now().time()  #time object
    pywhatkit.sendwhatmsg("", f"ERROR - {e}", time_hour=now.hour, time_min=now.minute+2)

2022-04-30 00:00:00: 100%|██████████| 269/269 [33:50:55<00:00, 452.99s/it]   


In [2]:
start_date

NameError: name 'start_date' is not defined

In [ ]:
end_date

In [9]:
filters = ['author', 'created_utc', "score", "body"]

In [8]:
timestamp_list

[1619820000,
 1619906400,
 1619992800,
 1620079200,
 1620165600,
 1620252000,
 1620338400,
 1620424800,
 1620511200,
 1620597600,
 1620684000,
 1620770400,
 1620856800,
 1620943200,
 1621029600,
 1621116000,
 1621202400,
 1621288800,
 1621375200,
 1621461600,
 1621548000,
 1621634400,
 1621720800,
 1621807200,
 1621893600,
 1621980000,
 1622066400,
 1622152800,
 1622239200,
 1622325600,
 1622412000,
 1622498400,
 1622584800,
 1622671200,
 1622757600,
 1622844000,
 1622930400,
 1623016800,
 1623103200,
 1623189600,
 1623276000,
 1623362400,
 1623448800,
 1623535200,
 1623621600,
 1623708000,
 1623794400,
 1623880800,
 1623967200,
 1624053600,
 1624140000,
 1624226400,
 1624312800,
 1624399200,
 1624485600,
 1624572000,
 1624658400,
 1624744800,
 1624831200,
 1624917600,
 1625004000,
 1625090400,
 1625176800,
 1625263200,
 1625349600,
 1625436000,
 1625522400,
 1625608800,
 1625695200,
 1625781600,
 1625868000,
 1625954400,
 1626040800,
 1626127200,
 1626213600,
 1626300000,
 1626386400,

In [10]:
posts = data_prep_comments(api, "wallstreetbets", start_time=1619820000, end_time=1619906400, filters=filters)

In [11]:
posts

[{'author': 'Archdemon2212',
  'body': 'Due diligence',
  'created_utc': 1619822878,
  'score': 1},
 {'author': 'JimCramersCoke',
  'body': 'ocean water',
  'created_utc': 1619822871,
  'score': 4},
 {'author': 'Overdue_bills',
  'body': 'Puts on Monday.',
  'created_utc': 1619822869,
  'score': 1},
 {'author': 'Horror-Lemon3200',
  'body': 'Welcome! You found your home',
  'created_utc': 1619822867,
  'score': 2},
 {'author': 'get_MEAN_yall',
  'body': 'I can trade until 8pm eastern. What broker do you use?',
  'created_utc': 1619822866,
  'score': 0},
 {'author': 'dulkion',
  'body': 'And here I am preaching the GME gospel at Thanksgiving, and not buying in until January.  \n\nI need to listen to me more often.',
  'created_utc': 1619822866,
  'score': 2},
 {'author': 'SnooCauliflowers4003',
  'body': 'its the best public college system in the country?',
  'created_utc': 1619822864,
  'score': 1},
 {'author': 'Forevergogo',
  'body': "Oh man, I'd be selling TSLA so fast.  Everyone kn

In [14]:
posts

[{'author': 'Archdemon2212',
  'body': 'Due diligence',
  'created_utc': 1619822878,
  'score': 1},
 {'author': 'JimCramersCoke',
  'body': 'ocean water',
  'created_utc': 1619822871,
  'score': 4},
 {'author': 'Overdue_bills',
  'body': 'Puts on Monday.',
  'created_utc': 1619822869,
  'score': 1},
 {'author': 'Horror-Lemon3200',
  'body': 'Welcome! You found your home',
  'created_utc': 1619822867,
  'score': 2},
 {'author': 'get_MEAN_yall',
  'body': 'I can trade until 8pm eastern. What broker do you use?',
  'created_utc': 1619822866,
  'score': 0},
 {'author': 'dulkion',
  'body': 'And here I am preaching the GME gospel at Thanksgiving, and not buying in until January.  \n\nI need to listen to me more often.',
  'created_utc': 1619822866,
  'score': 2},
 {'author': 'SnooCauliflowers4003',
  'body': 'its the best public college system in the country?',
  'created_utc': 1619822864,
  'score': 1},
 {'author': 'Forevergogo',
  'body': "Oh man, I'd be selling TSLA so fast.  Everyone kn

In [15]:
pd.DataFrame(posts)

,author,body,created_utc,score
0,Archdemon2212,Due diligence,1619822878,1
1,JimCramersCoke,ocean water,1619822871,4
2,Overdue_bills,Puts on Monday.,1619822869,1
3,Horror-Lemon3200,Welcome! You found your home,1619822867,2
4,get_MEAN_yall,I can trade until 8pm eastern. What broker do ...,1619822866,0
...,...,...,...,...
95,fpcane305,Cool story Bro....lol,1619822679,1
96,fkgaslighters,This video made my day lmao,1619822679,1
97,Top_Caramel5267,Worlds are colliding,1619822674,2
98,letsgetapplebees,I like to show up on zoom in my pjs and beanie,1619822671,1
